## CNN

一些注意点：
- padding的方式分为两种，'valid'和'same'
- valid采用丢弃的方式，多余元素去掉
- same采用补全的方式，按左奇右偶的方式补0


In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf
sess = tf.Session()

import numpy as np

### 1. 导入数据

In [2]:
#用tensorflow 导入数据
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./data/MNIST_data', one_hot = True)
print(mnist.test.labels.shape)
print(mnist.train.labels.shape)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz
(10000, 10)
(55000, 10)


### 2. 构建网络

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')

In [4]:
X_input = tf.placeholder(tf.float32, [None, 784])
y_input = tf.placeholder(tf.float32, [None, 10])

#把x转化为卷积所需要的形式
X = tf.reshape(X_input, [-1, 28, 28, 1])
#第一层卷积:5*5*1卷积核32个[5,5,1,32]
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)
#第一个pooling层
h_pool1 = max_pool_2x2(h_conv1)

#第二层卷积:5*5*32卷积核64个[5,5,32,64]
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = weight_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

#fc1
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 =bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#dropout 输出维度和h_fc1一样只是随机部分值为0
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob=keep_prob)

#输出层
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_pred = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [5]:
print('X_input:', X_input)
print('X:      ', X)
print('y_input:', y_input)
print('h_conv1:', h_conv1)
print('h_pool1:', h_pool1)
print('h_conv2:', h_conv2)
print('h_pool2:', h_pool2)
print('h_fc1:  ', h_fc1)
print('y_pred: ', y_pred)

X_input: Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
X:       Tensor("Reshape:0", shape=(?, 28, 28, 1), dtype=float32)
y_input: Tensor("Placeholder_1:0", shape=(?, 10), dtype=float32)
h_conv1: Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
h_pool1: Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32)
h_conv2: Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
h_pool2: Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
h_fc1:   Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)
y_pred:  Tensor("Softmax:0", shape=(?, 10), dtype=float32)


### 3.训练和预估

In [8]:
cross_entropy = -tf.reduce_sum(y_input *tf.log(y_pred))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_input, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
sess.run(tf.global_variables_initializer())

for i in range(5000):
    X_batch, y_batch = mnist.train.next_batch(batch_size=100)
    cost, acc, _ =sess.run([cross_entropy, accuracy, train_step], feed_dict={X_input:X_batch, y_input:y_batch, keep_prob:0.5})
    if (i+1) % 500 == 0:
        test_cost, test_acc = sess.run([cross_entropy, accuracy], feed_dict={X_input: mnist.test.images, y_input: mnist.test.labels, keep_prob: 1.0})
        print("step {}, train cost={:.6f}, acc={:.6f}; test cost={:.6f}, acc={:.6f}".format(i+1, cost, acc, test_cost, test_acc))

step 500, train cost=15.128507, acc=0.950000; test cost=1547.071533, acc=0.955800
step 1000, train cost=15.411701, acc=0.950000; test cost=958.668884, acc=0.970000
step 1500, train cost=12.403200, acc=0.970000; test cost=774.611023, acc=0.975700
step 2000, train cost=4.158594, acc=0.980000; test cost=591.042786, acc=0.980800
step 2500, train cost=13.705905, acc=0.940000; test cost=496.175629, acc=0.983200
step 3000, train cost=5.975717, acc=0.990000; test cost=442.569336, acc=0.985300
step 3500, train cost=6.144988, acc=0.980000; test cost=420.727844, acc=0.986600
step 4000, train cost=9.482490, acc=0.990000; test cost=377.285553, acc=0.986900
step 4500, train cost=3.740895, acc=0.990000; test cost=348.553772, acc=0.988300
step 5000, train cost=11.455607, acc=0.980000; test cost=326.203430, acc=0.988800
